In [1]:
from toolz import compose, first, juxt
from toolz.curried import map as mapc
from operator import attrgetter as at
from operator import itemgetter as it
from operator import methodcaller as mc
from operator import eq
from functools import partial
import polars as pl
from cuallee import Check

from Bio import SeqIO
from Bio.Seq import translate

import warnings
warnings.filterwarnings(action="ignore")


#filename = "test/fixtures/unextected_features/FR823450_1.gb"
filename = "temp/T01.gb"
genome = SeqIO.read(filename, "genbank")


cds_qualifiers = ("gene", "locus_tag", "protein_id", "function", "product", "translation", "transl_table", "codon_start")
gene_qualifiers = ("gene", "locus_tag")
location_attributes = ("start", "end", "strand")

annotations_attributes = ("topology", "organism", "taxonomy")
topology, organism, taxonomy = compose(juxt(map(it, annotations_attributes)), at("annotations"))(genome)

id_attributes = ("id", "name", "description")
id, name, description = juxt(map(at, id_attributes))(genome)

_impute_attributes = lambda x: mc("get", x, [""])

_type_cds = compose(partial(eq, "CDS"), at("type"))
_type_gene = compose(partial(eq, "gene"), at("type"))

data_cds = list(map(compose(list, mapc(first), juxt(map(_impute_attributes, cds_qualifiers)), at("qualifiers")), filter(_type_cds, genome.features)))
data_cds_pk = list(map(compose(list, mapc(at("real")), juxt(map(at, location_attributes)), at("location")), filter(_type_cds, genome.features)))

df_cds = pl.DataFrame(data_cds, schema=cds_qualifiers)
df_cds_pk = pl.DataFrame(data_cds_pk, schema=location_attributes)
cds_extract = list(map(lambda x: str(x.extract(genome.seq)), filter(_type_cds, genome.features)))
df_cds_extract = pl.DataFrame(cds_extract, schema=["extract"])
df_cds_translate = pl.DataFrame(map(partial(translate, stop_symbol="", table=11), cds_extract), schema=["translation_fn"])
cds = pl.concat(items=[df_cds, df_cds_pk, df_cds_extract, df_cds_translate], how="horizontal").rename(mapping={"gene": "cds_gene", "locus_tag": "cds_locus_tag"})


data_gene = list(map(compose(list, mapc(first), juxt(map(_impute_attributes, gene_qualifiers)), at("qualifiers")), filter(_type_gene, genome.features)))
data_gene_pk = list(map(compose(list, mapc(at("real")), juxt(map(at, location_attributes)), at("location")), filter(_type_gene, genome.features)))

df_gene = pl.DataFrame(data_gene, schema=gene_qualifiers)
df_gene_pk = pl.DataFrame(data_gene_pk, schema=location_attributes)
gene_extract = list(map(lambda x: str(x.extract(genome.seq)), filter(_type_gene, genome.features)))
df_gene_translate = pl.DataFrame(map(partial(translate, stop_symbol="", table=11), gene_extract), schema=["translation_fn"])
df_gene_extract = pl.DataFrame(gene_extract, schema=["extract"])

gene = pl.concat(items=[df_gene, df_gene_pk, df_gene_extract, df_gene_translate], how="horizontal")


df = cds.join(other=gene, on=["start", "end", "strand", "extract"], how="full", coalesce=True)
df = df.with_columns(
    id=pl.lit(id),
    name=pl.lit(name),
    description=pl.lit(description),
    topology=pl.lit(topology),
    organism=pl.lit(organism),
    taxonomy=pl.lit(taxonomy),
    filename=pl.lit(filename),
).drop_nulls()


# Create fasta file
check = Check()
check_columns = ["gene", "locus_tag", "cds_gene", "cds_locus_tag", "protein_id"]
[check.is_complete(c) for c in check_columns]
[check.is_unique(c) for c in check_columns]
check.validate(df)

id,timestamp,check,level,column,rule,value,rows,violations,pass_rate,pass_threshold,status
i64,str,str,str,str,str,str,i64,i64,f64,f64,str
1,"""2024-06-16 16:12:17""","""cuallee.check""","""WARNING""","""gene""","""is_complete""","""N/A""",4082,0,1.0,1.0,"""PASS"""
2,"""2024-06-16 16:12:17""","""cuallee.check""","""WARNING""","""locus_tag""","""is_complete""","""N/A""",4082,0,1.0,1.0,"""PASS"""
3,"""2024-06-16 16:12:17""","""cuallee.check""","""WARNING""","""cds_gene""","""is_complete""","""N/A""",4082,0,1.0,1.0,"""PASS"""
4,"""2024-06-16 16:12:17""","""cuallee.check""","""WARNING""","""cds_locus_tag""","""is_complete""","""N/A""",4082,0,1.0,1.0,"""PASS"""
5,"""2024-06-16 16:12:17""","""cuallee.check""","""WARNING""","""protein_id""","""is_complete""","""N/A""",4082,0,1.0,1.0,"""PASS"""
6,"""2024-06-16 16:12:17""","""cuallee.check""","""WARNING""","""gene""","""is_unique""","""N/A""",4082,496,0.878491,1.0,"""FAIL"""
7,"""2024-06-16 16:12:17""","""cuallee.check""","""WARNING""","""locus_tag""","""is_unique""","""N/A""",4082,0,1.0,1.0,"""PASS"""
8,"""2024-06-16 16:12:17""","""cuallee.check""","""WARNING""","""cds_gene""","""is_unique""","""N/A""",4082,496,0.878491,1.0,"""FAIL"""
9,"""2024-06-16 16:12:17""","""cuallee.check""","""WARNING""","""cds_locus_tag""","""is_unique""","""N/A""",4082,0,1.0,1.0,"""PASS"""


In [3]:
import uuid